In [274]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [275]:
#importing data
data = pd.read_csv('Desktop\dataset.csv')
data.head()

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,SERVER,CONTENT_LENGTH,WHOIS_COUNTRY,WHOIS_STATEPRO,WHOIS_REGDATE,WHOIS_UPDATED_DATE,...,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
0,M0_109,16,7,iso-8859-1,nginx,263.0,NaN,NaN,10-10-2015 18:21,NaN,...,0,2,700,9,10,1153,832,9,2.0,1
1,B0_2314,16,6,UTF-8,Apache/2.4.10,15087.0,NaN,NaN,NaN,NaN,...,7,4,1230,17,19,1265,1230,17,0.0,0
2,B0_911,16,6,us-ascii,Microsoft-HTTPAPI/2.0,324.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0
3,B0_113,17,6,ISO-8859-1,nginx,162.0,US,AK,07-10-1997 04:00,12-09-2013 00:45,...,22,3,3812,39,37,18784,4380,39,8.0,0
4,B0_403,17,6,UTF-8,NaN,124140.0,US,TX,12-05-1996 00:00,11-04-2017 00:00,...,2,5,4278,61,62,129889,4586,61,4.0,0


In [276]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 21 columns):
URL                          1781 non-null object
URL_LENGTH                   1781 non-null int64
NUMBER_SPECIAL_CHARACTERS    1781 non-null int64
CHARSET                      1774 non-null object
SERVER                       1605 non-null object
CONTENT_LENGTH               969 non-null float64
WHOIS_COUNTRY                1475 non-null object
WHOIS_STATEPRO               1413 non-null object
WHOIS_REGDATE                1652 non-null object
WHOIS_UPDATED_DATE           1642 non-null object
TCP_CONVERSATION_EXCHANGE    1781 non-null int64
DIST_REMOTE_TCP_PORT         1781 non-null int64
REMOTE_IPS                   1781 non-null int64
APP_BYTES                    1781 non-null int64
SOURCE_APP_PACKETS           1781 non-null int64
REMOTE_APP_PACKETS           1781 non-null int64
SOURCE_APP_BYTES             1781 non-null int64
REMOTE_APP_BYTES             1781 non-null int64
APP

In [277]:
#Number of null values
data.isnull().sum()

URL                            0
URL_LENGTH                     0
NUMBER_SPECIAL_CHARACTERS      0
CHARSET                        7
SERVER                       176
CONTENT_LENGTH               812
WHOIS_COUNTRY                306
WHOIS_STATEPRO               368
WHOIS_REGDATE                129
WHOIS_UPDATED_DATE           139
TCP_CONVERSATION_EXCHANGE      0
DIST_REMOTE_TCP_PORT           0
REMOTE_IPS                     0
APP_BYTES                      0
SOURCE_APP_PACKETS             0
REMOTE_APP_PACKETS             0
SOURCE_APP_BYTES               0
REMOTE_APP_BYTES               0
APP_PACKETS                    0
DNS_QUERY_TIMES                1
Type                           0
dtype: int64

In [278]:
#Some new columns which will be created by get dummies
print(len(data['WHOIS_COUNTRY'].unique().tolist()))
print(len(data['WHOIS_STATEPRO'].unique().tolist()))
print(len(data['CHARSET'].unique().tolist()))
print(len(data['SERVER'].unique().tolist()))

48
168
9
239


In [279]:
#Dropping URL because it is unique to each row and wouldn't add value to the model
df = data.drop('URL',axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 20 columns):
URL_LENGTH                   1781 non-null int64
NUMBER_SPECIAL_CHARACTERS    1781 non-null int64
CHARSET                      1774 non-null object
SERVER                       1605 non-null object
CONTENT_LENGTH               969 non-null float64
WHOIS_COUNTRY                1475 non-null object
WHOIS_STATEPRO               1413 non-null object
WHOIS_REGDATE                1652 non-null object
WHOIS_UPDATED_DATE           1642 non-null object
TCP_CONVERSATION_EXCHANGE    1781 non-null int64
DIST_REMOTE_TCP_PORT         1781 non-null int64
REMOTE_IPS                   1781 non-null int64
APP_BYTES                    1781 non-null int64
SOURCE_APP_PACKETS           1781 non-null int64
REMOTE_APP_PACKETS           1781 non-null int64
SOURCE_APP_BYTES             1781 non-null int64
REMOTE_APP_BYTES             1781 non-null int64
APP_PACKETS                  1781 non-null int64
DNS_

In [280]:
#Getting year from the datetime object 
import dateutil.parser as parser
regdate = []
for i in df['WHOIS_REGDATE']:
    if type(i) != type(""):
        regdate.append(None)
    else:
        try : 
            regdate.append(parser.parse(i).year)
        except :
            regdate.append(None)

df['WHOIS_REGDATE'] = regdate

update = []
for i in df['WHOIS_UPDATED_DATE']:
    if type(i) != type(""):
        update.append(None)
    else:
        try : 
            update.append(parser.parse(i).year)
        except :
            update.append(None)

df['WHOIS_UPDATED_DATE'] = update

In [281]:
new_df = pd.get_dummies(df,columns= ['CHARSET','WHOIS_COUNTRY','WHOIS_STATEPRO','SERVER'])
print(new_df[new_df.columns].shape)
print(new_df.columns)

(1781, 476)
Index(['URL_LENGTH', 'NUMBER_SPECIAL_CHARACTERS', 'CONTENT_LENGTH',
       'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE', 'TCP_CONVERSATION_EXCHANGE',
       'DIST_REMOTE_TCP_PORT', 'REMOTE_IPS', 'APP_BYTES', 'SOURCE_APP_PACKETS',
       ...
       'SERVER_nginx/1.8.0', 'SERVER_nginx/1.8.1', 'SERVER_nginx/1.9.13',
       'SERVER_nxfps', 'SERVER_openresty', 'SERVER_openresty/1.11.2.1',
       'SERVER_openresty/1.11.2.2', 'SERVER_squid/3.3.8', 'SERVER_tsa_c',
       'SERVER_www.lexisnexis.com  9999'],
      dtype='object', length=476)


In [282]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [283]:
X = new_df.drop('Type', axis = 1)

In [291]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X)
X = imp.transform(X)
print(X, type(X))

[[1.60000000e+01 7.00000000e+00 2.63000000e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.60000000e+01 6.00000000e+00 1.50870000e+04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.60000000e+01 6.00000000e+00 3.24000000e+02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.01000000e+02 3.40000000e+01 8.90400000e+03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.34000000e+02 3.40000000e+01 1.17269278e+04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.49000000e+02 4.00000000e+01 2.44350000e+04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]] <class 'numpy.ndarray'>


In [285]:
Y = new_df['Type']

In [286]:
#Splitting Training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=2)
# print the size of the traning set:
print(x_train.shape)
print(y_train.shape)

# print the size of the testing set:
print(x_test.shape)
print(y_test.shape)

(1246, 475)
(1246,)
(535, 475)
(535,)


In [287]:
#Building three models to compare results 
dec_tree=DecisionTreeClassifier() 
log_reg=LogisticRegression()
k=5
knn=KNeighborsClassifier(n_neighbors=k)

In [288]:
log_reg.fit(x_train,y_train)
dec_tree.fit(x_train,y_train)
knn.fit(x_train,y_train)

C:\Users\mbacchu\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [289]:
y_perdict_regression =log_reg.predict(x_test)
y_perdict_tree =dec_tree.predict(x_test)
y_predict_knn= knn.predict(x_test)

In [290]:
score_regression=accuracy_score(y_test,y_perdict_regression)
score_tree=accuracy_score(y_test, y_perdict_tree)
score_knn=accuracy_score(y_test, y_predict_knn)

print('Logistic Regression',score_regression)
print('Decision Tree',score_tree)
print('KNN',score_knn)

Logistic Regression 0.9401869158878504
Decision Tree 0.9345794392523364
KNN 0.9271028037383178


In [292]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
# specify model (xgboost defaults are generally fine)
model = xgb.XGBRegressor()

# fit our model
model.fit(y=Y, X=X)

C:\Users\mbacchu\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\mbacchu\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[13:58:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [313]:
# predictions & actual values, from test set
predictions = model.predict(x_test) > 0
actual = y_test
accuracy = accuracy_score(y_test,predictions)
print(accuracy)

0.4355140186915888


In [304]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
# plot confusion matrix 
confusion_matrix(actual, predictions)

array([[175, 302],
       [  0,  58]], dtype=int64)